In [ ]:
import shutil
import os,glob

In [ ]:
def find_district_directory(dummy_data_dir):
    
    return [i for i in next(os.walk(dummy_data_dir))[1]][0]


def find_road_directories(district):

    return [i for i in (glob.glob(district + os.sep + 'roads' + os.sep + '*')) if os.path.isdir(i)]

def split_roads_among_pcs(roads_list, num_pcs):
    """
    Split entire roads (without splitting segments) among PCs.
    :param roads_list: List of roads (paths) to distribute.
    :param num_pcs: Number of PCs.
    :return: List of lists where each sublist represents roads assigned to a specific PC.
    """
    # Sort roads by the number of segments (descending) to better distribute heavy roads first
    sorted_roads = sorted(roads_list, key=lambda x: len(os.listdir(os.path.join(x, 'segments'))), reverse=True)
    
    # Initialize lists to hold road assignments for each PC and track the segment count for each PC
    pcs = [[] for _ in range(num_pcs)]
    pc_segments = [0] * num_pcs
    
    # Assign each road to the PC with the least number of segments so far
    for road in sorted_roads:
        segments_count = len(os.listdir(os.path.join(road, 'segments')))
        min_pc = pc_segments.index(min(pc_segments))  # Find the PC with the fewest segments
        pcs[min_pc].append((road, segments_count))  # Assign the road and its segment count to this PC
        pc_segments[min_pc] += segments_count  # Update the segment count for the PC
    
    return pcs

def copy_road_to_pc_folders(pcs_allocation, base_path):
    """
    Copy the entire road folder (including the 'segments' folder and any additional files) 
    to the corresponding PC folder. Also, print the number of roads and segments in each PC.
    :param pcs_allocation: List of lists, with roads allocated to each PC.
    :param base_path: Base path where PC folders will be created.
    """
    for pc_num, roads in enumerate(pcs_allocation):
        pc_folder = os.path.join(base_path, f'PC_{pc_num + 1}')
        os.makedirs(pc_folder, exist_ok=True)
        
        total_segments = 0  # To track total segments for this PC
        print(f"PC_{pc_num + 1}:")
        
        for road, segments_count in roads:
            # Road directory path (the entire road folder, not just 'segments')
            road_dst = os.path.join(pc_folder+os.sep+'District'+os.sep+"roads", os.path.basename(road))
            
            # Copy the entire road folder (including files and 'segments' subfolder)
            try:
                shutil.copytree(road, road_dst)
                # print(f"  - Copied entire road '{os.path.basename(road)}' with {segments_count} segments.")
                total_segments += segments_count
            except Exception as e:
                print(f"Failed to copy {road} to {road_dst}: {e}")
        
        print(f"Total roads: {len(roads)}, Total segments: {total_segments}\n")

In [ ]:
num_pcs = 3

main_dir = 'Data'+ os.sep +'C&S related'+ os.sep +'dummy report data'
distt = find_district_directory(main_dir)
distt_path = main_dir + os.sep + distt
base_roads_folder = distt_path + os.sep + 'roads'
base_path = 'Data'+ os.sep +'C&S related'+ os.sep +'segregated data'

# Get the list of road directories
roads_list = [os.path.join(base_roads_folder, road) for road in os.listdir(base_roads_folder) if os.path.isdir(os.path.join(base_roads_folder, road))]

# Step 1: Split roads among PCs without splitting a road between PCs
pcs_allocation = split_roads_among_pcs(roads_list, num_pcs)

# Step 2: Copy 'segments' to corresponding PC folders and print stats
copy_road_to_pc_folders(pcs_allocation, base_path)

print('------  DONE  -------')
